In [7]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt

# Explicit Multistep Time Integration

In this notebook we look to replicate the multistep time integration for two-dimensional heterogeneous solids from the following paper

Reference: Cho, S. S., Kolman, R., González, J. A., & Park, K. C. (2019). Explicit multistep time integration for discontinuous elastic stress wave propagation in heterogeneous solids. International Journal for Numerical Methods in Engineering, 118(5), 276-302

## List of Symbols
- $\mathbf{B}_i$: Boolean matrices extracting interfaces DOFs for each domain
- $f$: frame
- $i$: Subdomain (either L or S)
- $k$: Order of the sub-step in the S-domain during a time step in the L-domain
- $\mathbf{L}_i$: Boolean matrices relating interface DOFs to global acceleration and displacement
- $m$: Ratio of the two time steps (and is INTEGER)
- $N$: Number of Large time steps
- $j$: $k/m$ 
- $j_o$: $k-1/m$
- $\alpha$: Ratio of time step and the Critical time step
- $\beta$: Parameter relating $\alpha$ and $\theta$
- $\gamma$: ??
- $\lambda$: Interface reaction force (Lagrange Multiplier)
- $\theta$: "Pushforward pullback" parameter $ \in[0,1](\theta=1/2$ for averaged displacement and $\theta = 0$ for CDM)



Start of Main Step $n=1:N$ on L-Domain

Step 1: Prepare $(\mathbf{u}_L^n , \dot{\mathbf{u}}_L^n , \ddot{\mathbf{u}}_L^n)$

Start of Substep $k = 1:m$ on S-Domain

Step 2: Integrate Small-Domain

Prepare $(\mathbf{u}_S^{n+j_o} , \dot{\mathbf{u}}_S^{n+j_o} , \ddot{\mathbf{u}}_S^{n+j_o})$

where $j_o = \frac{k-1}{m}$ for $ k = 1,2,...,m$ the k-th integration sub-step in the S-domain 

Step 2.1.1 Compute $(\mathbf{\lambda}_S^{n+jC_s})$

$\begin{array}{l}
\Delta t \triangleq \Delta t_{C S} \\
\mathbf{u}_S^{n+j C_S}=\tilde{\mathbf{u}}_S^{n+j C_S}-\beta_{2 S}\left(\Delta t_{C S}\right)^2 \mathbf{M}_S^{-1} \mathbf{B}_S \boldsymbol{\lambda}_S^{n+j C_S} \\
\tilde{\mathbf{u}}_S^{n+j C_S}=\mathbf{u}_S^{n+j_o}+\Delta t_{C S} \dot{\mathbf{u}}_S^{n+j_o}+\beta_{1 S}\left(\Delta t_{C S}\right)^2 \ddot{\mathbf{u}}_S^{n+\left(j_o\right.}+\beta_{2 S}\left(\Delta t_{C S}\right)^2 \tilde{\mathbf{u}}_S^{n+j C_S} \\
\mathbf{u}_f^{n+j C_S}=\mathbf{u}_f^{n+j_o}+\Delta t_{C S} \dot{\mathbf{u}}_f^{n+j_o}+\frac{1}{2}\left(\Delta t_{C S}\right)^2 \ddot{\mathbf{u}}_f^{n+j_o} \\
\boldsymbol{\lambda}_S^{n+j C_S}=\left[\beta_{2 S}\left(\Delta t_{C S}\right)^2 \mathbf{B}_S^T \mathbf{M}_S^{-1} \mathbf{B}_S\right]^{-1}\left(\mathbf{B}_S^T \tilde{\mathbf{u}}_S^{n+j C_S}-\mathbf{L}_S \mathbf{u}_f^{n+j C_S}\right)
\end{array}$

In [8]:
# Small Interface Loads
dt_critical_S = 0.1
dt = dt_critical_S
disp_S = np.zeros(2)
unconstrained_disp_S = np.zeros(2)
disp_f_L = np.zeros(2)
lambda_S = np.zeros(2)


Step 2.1.2 Compute $(\mathbf{\lambda}_S^{n+jC_L})$

for $k = 1:$
$\begin{array}{l}
\Delta t \triangleq \Delta t_{C L} \\
\mathbf{u}_L^{n+C_L}=\tilde{\mathbf{u}}_L^{n+C_L}-\beta_{2 L}\left(\Delta t_{C L}\right)^2 \mathbf{M}_L^{-1} \mathbf{B}_L \boldsymbol{\lambda}_L^{n+C_L} \\
\tilde{\mathbf{u}}_L^{n+C_L}=\mathbf{u}_S^n+\Delta t_{C L} \dot{\mathbf{u}}_L^n+\beta_{1 L}\left(\Delta t_{C L}\right)^2 \ddot{\mathbf{u}}_L^{n)}+\beta_{2 L}\left(\Delta t_{C L}\right)^2 \tilde{\mathbf{u}}_L^{n+C_L} \\
\mathbf{u}_f^{n+C_L}=\mathbf{u}_f^n+\Delta t_{C L} \dot{\mathbf{u}}_f^n+\frac{1}{2}\left(\Delta t_{C L}\right)^2 \ddot{\mathbf{u}}_f^n \\
\boldsymbol{\lambda}_L^{n+C_L}=\left[\beta_{2 L}\left(\Delta t_{C L}\right)^2 \mathbf{B}_L^T \mathbf{M}_L^{-1} \mathbf{B}_L\right]^{-1}\left(\mathbf{B}_L^T \tilde{\mathbf{u}}_L^{n+C_L}-\mathbf{L}_L \mathbf{u}_f^{n+C_L}\right)
\end{array}$

for $k = 2:m:$
$\begin{array}{l}
\Delta t \triangleq(k-1) \Delta t_S \\
\mathbf{u}_L^{n+C_L}=\tilde{\mathbf{u}}_L^{n+C_L}-\beta_{2 L}\left(\Delta t_{C L}\right)^2 \mathbf{M}_L^{-1} \mathbf{B}_L \boldsymbol{\lambda}_L^{n+C_L} \\
\tilde{\mathbf{u}}_L^{n+C_L}=\mathbf{u}_L^n+\Delta t_{C L} \dot{\mathbf{u}}_L^n+\beta_{1 L}\left(\Delta t_{C L}\right)^2 \ddot{\mathbf{u}}_L^{n)}+\beta_{2 L}\left(\Delta t_{C L}\right)^2 \tilde{\mathbf{u}}_L^{n+C_L} \\
\mathbf{L}_S \mathbf{u}_f^{n+j}=\mathbf{B}_S^T \mathbf{u}_S^{n+j} \\
\boldsymbol{\lambda}_L^{n+C_L}=\left[\beta_{2 L}\left(\Delta t_{C L}\right)^2 \mathbf{B}_L^T \mathbf{M}_L^{-1} \mathbf{B}_L\right]^{-1}\left(\mathbf{B}_L^T \tilde{\mathbf{u}}_L^{n+C_L}-\mathbf{L}_L \mathbf{u}_f^{n+C_L}\right)
\end{array}$


In [9]:
# Large Interface Loads
dt_critical_L = 0.1
dt_L = dt_critical_L
disp_L = np.zeros(2)
unconstrained_disp_L = np.zeros(2)
disp_f_L = np.zeros(2)
lambda_L = np.zeros(2)

Step 2.1.3: Compute $(\mathbf{\lambda}_S^{n+j} , \mathbf{\lambda}_L^{n+j} , \ddot{\mathbf{u}}_f^{n+j})$ at the interface $\Gamma_I$

$\mathbf{\lambda}_S^{n+j} = (\mathbf{B}^T_S \mathbf{M}_S^{-1} \mathbf{B}_S)^{-1} [\mathbf{B}_S^T \tilde{\ddot{u}}_S^{n+j} - \mathbf{L}_S \mathbf{u}_f^{n+j}]$ (Eq.48)

$\left[\begin{array}{c}
\boldsymbol{\lambda}_L \\
\ddot{\mathbf{u}}_f
\end{array}\right]^{n+j}=\left[\begin{array}{cc}
\mathbf{B}_L^T \mathbf{M}_L^{-1} \mathbf{B}_L & \mathbf{L}_L \\
\mathbf{L}_L^T & -\mathbf{L}_S^T\left(\mathbf{B}_S^T \mathbf{M}_S^{-1} \mathbf{B}_S\right)^{-1} \mathbf{L}_S
\end{array}\right]^{-1}\left[\begin{array}{c}
\mathbf{B}_L^T \tilde{\ddot{\mathbf{u}}}_L^{n+j} \\
-\mathbf{L}_S^T\left(\mathbf{B}_S^T \mathbf{M}_S^{-1} \mathbf{B}_S\right)^{-1} \mathbf{B}_S^T \tilde{\mathbf{u}}_S^{n+j}
\end{array}\right]$

In [ ]:
# Initialise Frame Variables 
lambda_S = np.zeros(2)
lambda_L = np.zeros(2)
acc_f = np.zeros(2)

Step 2.2 Compute $(\dot{\mathbf{u}}_S^{n+j}, \ddot{\mathbf{u}}_S^{n+j})$

$\begin{array}{l}
\ddot{\mathbf{u}}_S^{n+j}=\tilde{\mathbf{u}}_S^{n+j}-\mathbf{M}_S^{-1} \mathbf{B}_S \boldsymbol{\lambda}_S^{n+j} \\
\dot{\mathbf{u}}_S^{n+j}=\dot{\mathbf{u}}_S^{n+(j-1)}+\Delta t_S\left\{(1-\gamma) \ddot{\mathbf{u}}_S^{n+(j-1)}+\gamma \ddot{\mathbf{u}}_S^{n+j}\right\}
\end{array}$

In [ ]:
# Initialise Small Velocities and Small Accelerations
vel_S = np.zeros(2)
acc_S = np.zeros(2)

Step 2.3 Update $(\mathbf{u}_f^{n+j} , \dot{\mathbf{u}}_f^{n+j})$ at the interface $\Gamma_I$

$\begin{array}{l}
\dot{\mathbf{u}}_f^{n+j}=\dot{\mathbf{u}}_f^{n+(j-1)}+\Delta t_S\left\{(1-\gamma) \ddot{\mathbf{u}}_f^{n+(j-1)}+\gamma \ddot{\mathbf{u}}_f^{n+j}\right\} \\
\mathbf{u}_f^{n+j}=\mathbf{u}_f^{n+(j-1)}+\Delta t_S \dot{\mathbf{u}}_f^{n+j}
\end{array}$

In [ ]:
# Initialise Frame displacements and velocities
vel_f = np.zeros(2)
disp_f = np.zeros(2)

# End of Substep on S-Domain

Step 3: Integrate Large-Domain

Step 3.1: Compute $(\dot{\mathbf{u}}_L^{n+1} , \ddot{\mathbf{u}}_L^{n+1})$ on entire L-Domain

$\begin{array}{l}
\ddot{\mathbf{u}}_L^{n+1}=\tilde{\mathbf{u}}_L^{n+1}-\mathbf{M}_L^{-1} \mathbf{B}_L \boldsymbol{\lambda}_L^{n+1} \\
\dot{\mathbf{u}}_L^{n+1}=\dot{\mathbf{u}}_L^n+\Delta t_L\left\{(1-\gamma) \ddot{\mathbf{u}}_L^n+\gamma \ddot{\mathbf{u}}_L^{n+1}\right\}
\end{array}$

In [ ]:
# Initialise Large Accelerations and Large Velocities
accel_L = np.zeros(2)
vel_L = np.zeros(2)

Step 3.2 Update $(\mathbf{u}_L^{n+1} , \dot{\mathbf{u}}_L^{n+1})$ only at the interface $\Gamma_I$ to prevent drifting of the interface

$\mathbf{L}_L \mathbf{u}_L^{n+1}=\mathbf{B}_L^T \mathbf{u}_f^{n+1}$

$\mathbf{L}_L \dot{\mathbf{u}}_L^{n+1}=\mathbf{B}_L^T \dot{\mathbf{u}}_f^{n+1}$

In [ ]:
# Enforce continuity of displacements and velocities
disp_L = np.zeros(2)
vel_L = np.zeros(2)

# End of Main Step on L-Domain